## Setup the Python Environment

In [ ]:
! module use /g/data3/hh5/public/modules/
! module load conda/analysis3

## Import python libraries

In [ ]:
import xarray
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime
%matplotlib inline

## Define the files for calculating the climatology
The below options provide a list of the models, variables, experiments and ensembles that you want to calculate climatological fields for to use with the PMP analysis. The start and end year of the climatology that is calculated is also defined. Note that it is expected that the model output used here is monthly mean data.

In [ ]:
models = ['ACCESS1-0', 'ACCESS1-3','IPSL-CM5A-LR','CanESM2','CanAM4','CMCC-CESM','CNRM-CM5','FGOALS-g2','MIROC5','HadCM3','MPI-ESM-LR','MRI-ESM1','GISS-E2-H','CCSM4','NorESM1-M','GFDL-CM2p1']  #which models do you want to include in the analysis
variables = ['tas','pr','sos','tos','sic','sit','snd']             #which variables
variable_realm = ['atmos','atmos','ocean','ocean','seaIce','seaIce','landIce']   #the realm must match that for the selected variable
experiments = ['historical']             #which experiments
ensemble = ['r1i1p1']                #which ensemble
year_start = 1901                     #what is the start year for the climatology to be calculated
year_end = 1950                       #what is the end year for the climatology to be calculated

Determine the appropriate institue and realm given the model options.

In [ ]:
table_id = {'atmos':'Amon','ocean':'Omon','seaIce':'OImon','land':'Lmon','landIce':'LImon','aerosol':'aero'}
institute = {'ACCESS1-0':'CSIRO-BOM','ACCESS1-3':'CSIRO-BOM','CSIRO-Mk3-6-0':'UNSW','CSIRO-Mk3L-1-2':'UNSW','CanAM4':'CCCma',\
             'CanCM4':'CCCma','CanESM2':'CCCma','CMCC-CESM':'CMCC','CMCC-CM':'CMCC','CMCC-CMS':'CMCC','CNRM-CM5':'CNRM-CERFACS',\
             'CNRM-CM5-2':'CNRM-CERFACS','EC-EARTH':'ICHEC','inmcm4':'INM','IPSL-CM5A-LR':'IPSL','IPSL-CM5A-MR':'IPSL','IPSL-CM5B-LR':'IPSL',\
             'FGOALS-g2':'LASG-CESS','FGOALS-s2':'LASG-IAP','MIROC-ESM':'MIROC','MIROC-ESM-CHEM':'MIROC','MIROC4h':'MIROC',\
             'MIROC5':'MIROC','HadCM3':'MOHC','HadGEM2-A':'MOHC','HadGEM2-CC':'MOHC','HadGEM2-ES':'MOHC','MPI-ESM-LR':'MPI-M',\
             'MPI-ESM-MR':'MPI-M','MPI-ESM-P':'MPI-M','MRI-AGCM3-2H':'MRI','MRI-AGCM3-2S':'MRI','MRI-CGCM3':'MRI','MRI-ESM1':'MRI',\
             'GISS-E2-H':'NASA-GISS','GISS-E2-H-CC':'NASA-GISS','GISS-E2-R':'NASA-GISS','GISS-E2-R-CC':'NASA-GISS','GEOS-5':'NASA-GMAO',\
             'CCSM4':'NCAR','NorESM1-M':'NCC','NorESM1-ME':'NCC','HadGEM2-AO':'NIMR-KMA','GFDL-CM2p1':'NOAA-GFDL','GFDL-CM3':'NOAA-GFDL',\
             'GFDL-ESM2G':'NOAA-GFDL','GFDL-ESM2M':'NOAA-GFDL','GFDL-HIRAM-C180':'NOAA-GFDL','GFDL-HIRAM-C360':'NOAA-GFDL',\
             'CFSv2-2011':'NOAA-NCEP'\
            }

## Loop through the desired options to calculate each climatology file
Here we loop through each of the options specified above and individually calcaulte the climatologies over the specified time range, saving the climatology file as a netCDF output to the current working directory. These files are necessary to then proceed to use the PMP tool as it is designed in version v1.1


In [ ]:
for i in range(len(models)):
    for j in range(len(experiments)):
        for k in range(len(variables)):
            for l in range(len(ensemble)):
                #find the file path
                CMIP_dir = '%s/%s/%s/mon/%s/%s/%s'%(institute[models[i]],models[i],experiments[j],\
                                                    variable_realm[k],table_id[variable_realm[k]],ensemble[l])
                if 'ACCESS' in models[i] or 'CSIRO' in models[i]:
                    out_dir = 'rr3/publications/CMIP5/output1'
                    version = 'latest'
                else:
                    out_dir = 'al33/replicas/CMIP5/combined'
                    #list directories with the different versions and determine the latest version
                    version = os.listdir('/g/data/%s/%s'%(out_dir,CMIP_dir))[-1]
                                                                                                        
                path = '/g/data/%s/%s/%s/%s/*.nc'%(out_dir,CMIP_dir,version,variables[k])
                print(path)
                   
                #open the dataset
                f = xarray.open_mfdataset(path,decode_times=False)
                f2 = xarray.decode_cf(f)
             
                #select the data within the time range chosen
                date_start = datetime.datetime(year_start, 1, 1, 0, 0)
                date_end = datetime.datetime(year_end + 1, 1, 1, 0, 0)
                
                #Take average in time over selected years to calculate the climatology
                climatology=f2.sel(time=slice(date_start, date_end)).groupby('time.month').mean(['time'])

                #save to a netcdf file
                climatology.to_netcdf('clim_CMIP5_%s_%s_%s_%i_%i.nc'%(models[i],experiments[j],variables[k],\
                                                                      year_start,year_end))
                
                